In [15]:
# simple multiprocessing communication between 1 BS and 1 UE
# 14.11.2023 - author Septimia Sarbu, ICON, CWC, University of Oulu

import numpy as np
import torch
from torch import nn

from scipy.stats import bernoulli, expon, poisson
from scipy.stats import rv_discrete
import matplotlib.pyplot as plt

import pdb

import math

from numpy.random import randint, random

import multiprocessing as mp
from multiprocessing import Process, Pool, Lock, RLock, Semaphore, Event, Queue, Value

from queue import Empty

import os, sys
from time import sleep

from numpy import save, load

import gymnasium as gym


class UEModel():
    def __init__(self,idx,item,env):
        
        self.id = idx 
        self.msg = -1
        self.data = item
        self.s = item # the state of the UE; it starts full, having all the nSDUs it needs to transmit
        self.env = env
        
        self.seed = 48
        self.rng = np.random.default_rng(self.seed)
      
    def receive_msg(self,queue):
        self.msg = queue.get()

    def send_msg(self,queue,item):
        queue.put(item)

    def send_hold(self,queue):
        queue.put('H')
    
    def send_end(self,queue):
        queue.put(None)
        
class BSModel():
    def __init__(self,idx,nUEs):
        
        self.msg = [-1 for i in range(nUEs)]
        self.got_data = 0
        self.data = 0
        
        self.seed = 480
        self.rng = np.random.default_rng(self.seed)

    def send_msg(self,queue,item,nUEs,run_UEs):
        for i in run_UEs:
            queue[i].put(item[i])
        
    def receive_msg(self,queue,nUEs,run_UEs):
        self.msg = [queue[i].get() for i in run_UEs]

def runUE(UCM,DCM,ch,tSDUs,UEModel,idx):
    
    # redirect the print() to a file
    original_stdout = sys.stdout # Save a reference to the original standard output

    f = open('M_UE'+str(idx)+'.txt', 'w')

    sys.stdout = f # Change the standard output to the file we created
    
    print('UE: Running using gym env: HalfCheetah-v4', flush=True)
     
    env = gym.make("HalfCheetah-v4")

    UE = UEModel(idx,tSDUs,env)
    
    obs, info = UE.env.reset() 
    #print("UE CP state = ",UE.env.state)
    print("UE Mujoco state = ",obs)
    
    done_flag = 0
    
    UE.receive_msg(DCM[idx])
    
    tUE = 0 # at tUE=0 we have START
    
    if UE.msg == 'START':
        print('-----------------------------------------------------')
        print(f'>At UE-{idx} START of transmission', flush=True)
        print('-----------------------------------------------------')
        
        done_flag = 1
        
    while done_flag == 1:
        
        print('-----------------------------------------------------')
        tUE += 1
        print("tUE=",tUE)
        
        print("UE Mujoco state = ",obs)
        print("UE sends Mujoco state", flush=True)
        UE.send_msg(UCM[idx],obs)
        
        print('-----------------------------------------------------')
        tUE += 1
        print("tUE=",tUE)
        print("UE waits for msg from BS", flush=True)
        UE.receive_msg(DCM[idx])
        print(f'>UE received msg={UE.msg} from BS', flush=True)
        
        if UE.msg is None:
            done_flag = 0 # end of communication

        else:
            act_test = env.action_space.sample()
            print(act_test)
            print(act_test.shape)
            # UE receives the action from the BS
            action = UE.msg
            print("Received action is =", action)
            # UE executes the action received from the BS
            n_obs, r, terminated, truncated, info = env.step(action)
            
            print("UE executed action received from BS")
            print("UE next state = ",n_obs)
            if terminated or truncated:
                obs, info = UE.env.reset()
                #break
            else:
                obs = n_obs
            

    # all done
    tUE += 1
    print('UE: tUE=',tUE, flush=True)
    print('UE: waiting to send None to BS', flush=True)
    UE.send_end(UCM[idx])
    print(f'>UE-{idx}: Done, sent None to BS', flush=True)

    sys.stdout = original_stdout # Reset the standard output to its original value
    print(f'>UE-{idx}: Done', flush=True)
    print('UE: tUE=',tUE, flush=True)

def runBS(UCM,DCM,ch,BSModel,nUEs):
    
    # redirect the print() to a file
    original_stdout = sys.stdout # Save a reference to the original standard output

    f = open('M_BS.txt', 'w')

    sys.stdout = f # Change the standard output to the file we created

    print('BS: Running', flush=True)

    run_UEs = [i for i in range(nUEs)] # this list contains the idx of the UEs that are transmitting

    BS = BSModel(0,nUEs)
    
    tEND = 5
    tBS = 0
    print('-----------------------------------------------------')
    print("tBS=",tBS)
    value = ['START' for i in range(nUEs)]
    BS.send_msg(DCM,value,nUEs,run_UEs)
    print(f'>BS put {value} in DCM queue', flush=True)
    print('-----------------------------------------------------')

    done_flag = 1
    while done_flag == 1:
        
        tBS += 1
        print("tBS=",tBS)
        print("BS is waiting for msg from UE", flush=True)
        BS.receive_msg(UCM,nUEs,run_UEs)
        print(f'>BS received msg={BS.msg} from UE', flush=True)
        
        
        for idx in range(len(run_UEs)):
            if BS.msg[idx] is None:
                print(f'>NONE: BS received msg={BS.msg} from UE', flush=True)
                done_flag = 0
        
        if done_flag == 1:
            print('-----------------------------------------------------')
            tBS += 1

            if tBS >= tEND:
                print("tBS=",tBS)
                print("BS is sending None to UE")
                BS.send_msg(DCM,[None],nUEs,run_UEs)
            else:
                print("tBS=",tBS)
                
                # select action vector between [-1, 1]
                act = BS.rng.uniform(low=-1,high=1,size=6) 
        
                print(f'>BS is sending msg={act} to UE', flush=True)
                
                #BS.send_msg(DCM,["BS"],nUEs,run_UEs)
                BS.send_msg(DCM,[act],nUEs,run_UEs)

        print('-----------------------------------------------------')

    # all done
    print('BS: tBS=',tBS, flush=True)
    print('BS: Done', flush=True)
    print('BS has data =',BS.data, flush=True) 
    
    sys.stdout = original_stdout # Reset the standard output to its original value
    
    print('BS: tBS=',tBS, flush=True)
    print('BS: Done', flush=True)
    print('BS has data =',BS.data, flush=True) 


# entry point
if __name__ == '__main__':
        
    # create the UE and the BS
    tSDUs = 2 # SDUs to transmit from each UE to BS, then the episode ends

    nUEs = 1
    
    # create the shared queues
    UCM = [Queue() for i in range(nUEs)]
    DCM = [Queue() for i in range(nUEs)]
    
    ch = Queue()

    # start the UE
    UE_processes = [Process(target=runUE, args=(UCM,DCM,ch,tSDUs,UEModel,i,)) for i in range(nUEs)]
    for UE_process in UE_processes:
        UE_process.start()
    
    # start the BS
    BS_process = Process(target=runBS, args=(UCM,DCM,ch,BSModel,nUEs,))
    BS_process.start()
        
    BS_process.join()
    
    # wait for all processes to finish
    for UE_process in UE_processes:
        UE_process.join()

    print("All processes have terminated successfully")

>UE-0: Done
BS: tBS=UE: tUE=  77

BS: Done
BS has data = 0
All processes have terminated successfully
